In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# [TR-016] Complex integral

<!-- cspell:ignore ndim orthopy -->
<!-- cspell:ignore epsabs ename epsrel evalue infbounds kronrod limlst maxp1 pycode retval subintervals wvar wopts -->
<!-- cspell:ignoreRegExp m(_qagie|_qagse|args|def|else|epsabs|epsrel|full|func|integrand|integrate|limit|linspace|max|min|not|parametrized|partial|points|quad|return) -->

```{autolink-skip}
```

In [ ]:
%pip install -q ndim==0.1.7 orthopy==0.9.6 quadpy==0.16.11 scipy==1.8.0 sympy==1.10.1

Note: you may need to restart the kernel to use updated packages.


As noted in {doc}`/report/003`, {func}`scipy.integrate.quad` cannot handle complex integrals. In addition, one can get into trouble with vectorized input ({obj}`numpy.array`s) on a lambdified {class}`sympy.Integral <sympy.integrals.integrals.Integral>`. This report discusses both problems and proposes some solutions.

## Complex integration

SciPy cannot integrate complex functions:

In [ ]:
from scipy.integrate import quad


def integrand(x):
    return x * (x + 1j)


quad(integrand, 0.0, 2.0)

TypeError: can't convert complex to float

### Split real and imaginary integral

A [proposed solution](https://stackoverflow.com/a/5966088) is to wrap the {func}`~scipy.integrate.quad` function in a special integrate function that integrates the real and imaginary part of a function separately:

In [ ]:
import numpy as np


def complex_integrate(func, a, b, **quad_kwargs):
    def real_func(x):
        return np.real(func(x))

    def imag_func(x):
        return np.imag(func(x))

    real_integral, real_integral_err = quad(real_func, a, b, **quad_kwargs)
    imag_integral, imag_integral_err = quad(imag_func, a, b, **quad_kwargs)
    return (
        real_integral + 1j * imag_integral,
        real_integral_err**2 + 1j * imag_integral_err,
    )

In [ ]:
complex_integrate(integrand, 0.0, 2.0)

((2.666666666666667+2j), (8.765121169122355e-28+2.220446049250313e-14j))

:::{warning}

The handling of uncertainties is incorrect.

:::

### Integrate with `quadpy`

[Alternatively](https://stackoverflow.com/a/42866568), one could use [`quadpy`](https://github.com/sigma-py/quadpy), which essentially does the same as in {ref}`report/016:Split real and imaginary integral`, but can also (to a large degree) handle vectorized input and properly handles uncertainties.

In [ ]:
import quadpy

quadpy.quad(integrand, a=0.0, b=2.0)

((2.6666666666666665+2.0000000000000004j), 2.0082667671941473e-19)

:::{note}

One may need to play around with the tolerance if the function is not smooth, see [sigma-py/quadpy#255](https://github.com/sigma-py/quadpy/issues/255).

:::

:::{tip}

[`quadpy`](https://github.com/sigma-py/quadpy) raises exceptions with {obj}`ModuleNotFoundError`s that are a bit unreadable. They are caused by the fact that [`orthopy`](https://pypi.org/project/orthopy) and [`ndim`](https://pypi.org/project/ndim) need to be installed separately.

:::

## Vectorized input

The dispersion integral from Eq.&nbsp;{eq}`dispersion-integral` in {doc}`/report/003` features a variable&nbsp;$s$ that is an argument to the function $\Sigma_a$. This becomes a problem when $s$ gets vectorized (in this case: gets an event-wise {obj}`numpy.array` of invariant masses). Here's a simplified version of the problem:

In [ ]:
from functools import partial


def parametrized_func(s_prime, s):
    return s_prime * (s_prime + s + 1j)


s_array = np.linspace(-1, 1, num=10)
quadpy.quad(
    partial(parametrized_func, s=s_array),
    a=0.0,
    b=2.0,
)

ValueError: operands could not be broadcast together with shapes (21,) (10,) 

The way out seems to be to **vectorize the [`quadpy.quad()`](https://github.com/sigma-py/quadpy) call itself** and forward the function arguments through {func}`functools.partial`:

In [ ]:
from functools import partial


@np.vectorize
def vectorized_quad(func, a, b, **func_kwargs):
    return quadpy.quad(partial(func, **func_kwargs), a, b)


vectorized_quad(parametrized_func, a=0.0, b=2.0, s=s_array)

(array([0.66666667+2.j, 1.11111111+2.j, 1.55555556+2.j, 2.        +2.j,
        2.44444444+2.j, 2.88888889+2.j, 3.33333333+2.j, 3.77777778+2.j,
        4.22222222+2.j, 4.66666667+2.j]),
 array([1.94765926e-19, 2.69476631e-19, 2.24127752e-19, 2.79100064e-19,
        2.67216263e-19, 1.43065895e-19, 3.08910645e-19, 3.62329394e-19,
        4.86795288e-19, 2.21702097e-19]))

Note, however, that this becomes difficult to implement as {func}`~sympy.utilities.lambdify.lambdify` output for a {class}`sympy.Integral <sympy.integrals.integrals.Integral>`. An attempt at this is made in {doc}`/report/003`.

## SymPy integral

There is no good way to write integrals as [SymPy](https://docs.sympy.org) expressions that correctly {func}`~sympy.utilities.lambdify.lambdify` to a **vectorized** integral that handles complex values. Here is a first step however. Note that this integral expression class derives from {class}`sympy.Integral <sympy.integrals.integrals.Integral>` and:

1. overwrites its {meth}`~sympy.core.basic.Basic.doit` method, so that the integral cannot be evaluated by SymPy.
2. provides a custom NumPy printer method (see {doc}`/report/001`) that lambdifies this expression node to [`quadpy.quad()`](https://github.com/sigma-py/quadpy).
3. adds class variables that can affect the behavior of [`quadpy.quad()`](https://github.com/sigma-py/quadpy).

In [ ]:
import sympy as sp
from sympy.printing.pycode import _unpack_integral_limits


class UnevaluatableIntegral(sp.Integral):
    abs_tolerance = 1e-5
    rel_tolerance = 1e-5
    limit = 50

    def doit(self, **hints):
        args = [arg.doit(**hints) for arg in self.args]
        return self.func(*args)

    def _numpycode(self, printer, *args):
        integration_vars, limits = _unpack_integral_limits(self)
        if len(limits) != 1:
            raise ValueError(
                f"Cannot handle {len(limits)}-dimensional integrals"
            )
        integrate = "quadpy_quad"
        printer.module_imports["quadpy"].update({f"quad as {integrate}"})
        limit_str = "%s, %s" % tuple(map(printer._print, limits[0]))
        args = ", ".join(map(printer._print, integration_vars))
        expr = printer._print(self.args[0])
        return (
            f"{integrate}(lambda {args}: {expr}, {limit_str},"
            f" epsabs={self.abs_tolerance}, epsrel={self.abs_tolerance},"
            f" limit={self.limit})[0]"
        )

To test whether this works, we write the expression from {ref}`report/016:Vectorized input` as a {class}`sympy.Expr <sympy.core.expr.Expr>`. Note that the integral indeed does not evaluate when calling {meth}`~sympy.core.basic.Basic.doit`:

In [ ]:
s, s_prime, a, b = sp.symbols("s s_p a b")
integral_expr: sp.Expr = UnevaluatableIntegral(
    s_prime * (s_prime + s + sp.I),
    (s_prime, a, b),
)
integral_expr.doit()

Integral(s_p*(s + s_p + I), (s_p, a, b))

Indeed the expression correctly lambdifies correctly:

In [ ]:
import inspect

integral_func = sp.lambdify([s, a, b], integral_expr)
src = inspect.getsource(integral_func)
print(src)

def _lambdifygenerated(s, a, b):
    return quadpy_quad(lambda s_p: s_p*(s + s_p + 1j), a, b, epsabs=1e-05, epsrel=1e-05, limit=50)[0]



Note, however, that the lambdified function has to be vectorized before it can handle {obj}`numpy.array`s:

In [ ]:
vec_integral_func = np.vectorize(integral_func)
vec_integral_func(s_array, a=0.0, b=2.0)

array([0.66666667+2.j, 1.11111111+2.j, 1.55555556+2.j, 2.        +2.j,
       2.44444444+2.j, 2.88888889+2.j, 3.33333333+2.j, 3.77777778+2.j,
       4.22222222+2.j, 4.66666667+2.j])

:::{tip}

For a more complicated and challenging expression, see {ref}`report/003:SymPy expressions` in {doc}`/report/003`.

:::